# Exploring Airports in Australia

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint

# Import API key
from config import geoapify_key


## DataFrame Preparation

In [2]:
# Import cities file as DataFrame
city_df = pd.read_csv("../Resources/Cities.csv")
city_df

,City
0,Sydney
1,Melbourne
2,Perth
3,Cairns
4,Adelaide
5,Gold Coast
6,Townsville
7,Williamtown


In [3]:
# Add columns the airports data we will fetch using the Geoapify API
# Note that we used "" to specify initial entry.
city_df["Lat"] = ""
city_df["Lon"] = ""
city_df["Airport Name"] = ""
city_df["IATA Name"] = ""
city_df["Airport Address"] = ""
city_df["Distance"] = ""
city_df["Website"] = ""
city_df

,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,,,,,,,
1,Melbourne,,,,,,,
2,Perth,,,,,,,
3,Cairns,,,,,,,
4,Adelaide,,,,,,,
5,Gold Coast,,,,,,,
6,Townsville,,,,,,,
7,Williamtown,,,,,,,


## Look for Geographical Coordinates for Each City

In [4]:
# Define the API parameters
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

In [5]:
 params = {
    "apiKey":geoapify_key,
    "format":"json",
     "text": "Sydney, Australia"
}
response = requests.get(base_url, params = params).json()
pprint(response)

{'query': {'parsed': {'city': 'sydney',
                      'country': 'australia',
                      'expected_type': 'city'},
           'text': 'Sydney, Australia'},
 'results': [{'address_line1': 'Sydney',
              'address_line2': 'NSW, Australia',
              'bbox': {'lat1': -34.1732416,
                       'lat2': -33.3641864,
                       'lon1': 150.260825,
                       'lon2': 151.343898},
              'category': 'administrative',
              'city': 'Sydney',
              'country': 'Australia',
              'country_code': 'au',
              'datasource': {'attribution': '© OpenStreetMap contributors',
                             'license': 'Open Database License',
                             'sourcename': 'openstreetmap',
                             'url': 'https://www.openstreetmap.org/copyright'},
              'formatted': 'Sydney, NSW, Australia',
              'lat': -33.8698439,
              'lon': 151.2082848,
        

In [6]:
# Print a message to follow up the airport search
print("Starting airport search")

# Loop through the cities_pd DataFrame and search coordinates for each city
for index, row in city_df.iterrows():

    # Get the city's name & add ", Australia" to the string so geoapify finds the correct city
    city = row["City"] + ", Australia"

    # Add the current city to the parameters
    params = {
    "apiKey":geoapify_key,
    "format":"json",
    "text": str(city)
}
    print(city)
    # Make the API request
    response = requests.get(base_url, params = params).json()
    
    # Convert response to JSON
    

    # Extract latitude and longitude
    city_df.loc[index,"Lat"] = response["results"][0]["lat"]
    city_df.loc[index,"Lon"] = response["results"][0]["lon"]
    
    # Log the search results
    print(f"Coordinates for {city} fetched...")

# Display sample data to confirm that the coordinates appear
city_df

Starting airport search
Sydney, Australia
Coordinates for Sydney, Australia fetched...
Melbourne, Australia
Coordinates for Melbourne, Australia fetched...
Perth, Australia
Coordinates for Perth, Australia fetched...
Cairns, Australia
Coordinates for Cairns, Australia fetched...
Adelaide, Australia
Coordinates for Adelaide, Australia fetched...
Gold Coast, Australia
Coordinates for Gold Coast, Australia fetched...
Townsville, Australia
Coordinates for Townsville, Australia fetched...
Williamtown, Australia
Coordinates for Williamtown, Australia fetched...


,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,-33.869844,151.208285,,,,,
1,Melbourne,-37.814245,144.963173,,,,,
2,Perth,-31.955893,115.860585,,,,,
3,Cairns,-16.920666,145.772185,,,,,
4,Adelaide,-34.928181,138.599931,,,,,
5,Gold Coast,-28.002373,153.414599,,,,,
6,Townsville,-19.256939,146.823954,,,,,
7,Williamtown,-32.815,151.842778,,,,,


## Retrieve Airports' Information

In [15]:
params = {
    "categories":"airport",
    "apiKey": geoapify_key,
    "limit":20
}

params["filter"] = f"circle:{longitude},{latitude},{radius}"
params["bias"] = f"proximity:{longitude},{latitude}"

response = requests.get(base_url, params = params).json()
pprint(response)

{'features': [{'geometry': {'coordinates': [151.84238794840633,
                                            -32.806098849501566],
                            'type': 'Point'},
               'properties': {'address_line1': 'Newcastle Airport',
                              'address_line2': '55 Slades Road, Williamtown '
                                               'NSW 2318, Australia',
                              'airport': {'iata': 'NTL', 'icao': 'YWLM'},
                              'categories': ['airport',
                                             'airport.international',
                                             'internet_access',
                                             'internet_access.free'],
                              'city': 'Port Stephens Council',
                              'contact': {'phone': '+61 2 4928 9800'},
                              'country': 'Australia',
                              'country_code': 'au',
                              'dat

In [13]:
# Set parameters to search for airport's details
radius = 50000
params = {
    "categories":"airport",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the airport search
print("Starting airport details search")

# Iterate through the types_df DataFrame
for index, row in city_df.iterrows():
    
    # Get the city's name
    city = row["City"]
    
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lon"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
           
    # Make an API request using the params dictionary
    response = requests.get(base_url, params = params).json()
        
    # Convert the API response to JSON format

    
    # Grab the first airport from the results and store the details in the DataFrame
    try:
        city_df.loc[index, "Airport Name"] = response["features"][0]["properties"]["name"]
        city_df.loc[index, "IATA Name"] = response["features"][0]["properties"]["datasource"]["raw"]["iata"]
        city_df.loc[index, "Airport Address"] = response["features"][0]["properties"]["address_line2"]
        city_df.loc[index, "Distance"] = response["features"][0]["properties"]["distance"]
        city_df.loc[index, "Website"] = response["features"][0]["properties"]["datasource"]["raw"]["website"]
        

    except KeyError as e:
        # If no airport is found, log the error.
        print(f"{e.args[0]} not found for {city_df.loc[index, 'Airport Name']}")
        
    # Log the search results
    print(f"nearest airport from {city}: {city_df.loc[index, 'Airport Name']}")

# Display sample data
city_df

Starting airport details search
website not found for Rose Bay Seaplane Base
nearest airport from Sydney: Rose Bay Seaplane Base
nearest airport from Melbourne: Essendon Fields Airport
nearest airport from Perth: Perth Airport
website not found for Cairns Airport
nearest airport from Cairns: Cairns Airport
nearest airport from Adelaide: Adelaide Airport
nearest airport from Gold Coast: Southport Flying Club
nearest airport from Townsville: Townsville Airport
nearest airport from Williamtown: Newcastle Airport


,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,-33.869844,151.208285,Rose Bay Seaplane Base,RSE,"Rose Bay Cycleway, Rose Bay NSW 2029, Australia",5079,
1,Melbourne,-37.814245,144.963173,Essendon Fields Airport,MEB,"72 Hargrave Avenue, Essendon Fields VIC 3041, ...",9983,https://www.ef.com.au/
2,Perth,-31.955893,115.860585,Perth Airport,PER,"McEwing Road, Redcliffe WA 6104, Australia",8775,https://www.perthairport.com.au/
3,Cairns,-16.920666,145.772185,Cairns Airport,CNS,"Airport Avenue, Aeroglen QLD 4870, Australia",3461,
4,Adelaide,-34.928181,138.599931,Adelaide Airport,ADL,"Adelaide Airport SA 5950, Australia",4898,https://www.adelaideairport.com.au/
5,Gold Coast,-28.002373,153.414599,Southport Flying Club,SHQ,"Doug Bruhn Way, Gold Coast City QLD 4216, Aust...",9391,http://southportflyingclub.com.au/
6,Townsville,-19.256939,146.823954,Townsville Airport,TSV,"Gypsy Moth Court, Garbutt QLD 4810, Australia",4711,https://www.townsvilleairport.com.au/
7,Williamtown,-32.815,151.842778,Newcastle Airport,NTL,"55 Slades Road, Williamtown NSW 2318, Australia",620,https://www.newcastleairport.com.au


In [ ]:
# Save Data to CSV
